# Notebook with  MoveNet and YOLO

In [2]:
import torch, cv2
import numpy as np

import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [3]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom
# model settings:
model.classes = [0] # only detect humans -> class = 0

Using cache found in C:\Users\keven/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-4-22 torch 1.11.0 CUDA:0 (NVIDIA GeForce GTX 1050, 2048MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [6]:
def get_info_objs(results):
    return results.pandas().xyxy[0].sort_values('confidence')

In [7]:
def display_image(img):
    cv2.imshow('hello',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
def get_cropped_images(pd,frame): #pd is a 
    nb_detected_objs = len(pd.index)
    
    index = np.zeros((4,nb_detected_objs),dtype=int)#4 rows for x0,y0,x1,y1 --> each column is the bb of 1 img
    images = []
    
    counter = 0
    for obj in pd.iloc:
        x0, y0, x1, y1= obj.to_numpy()[0:4].astype(int)
        images.append(frame[y0:y1, x0:x1])
        index[:,counter] = np.array([x0,y0,x1,y1],dtype=int)
        counter += 1
        
    return images, index 

In [9]:
interpreter = tf.lite.Interpreter(model_path='C:\DLAV_testing\MoveNetLightning\lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [10]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [11]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [12]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [13]:
def write_frame_keypoints(imglist,indexes,frame):
    for i,objs in enumerate(imglist):    
        img = objs
        img1 = img.copy()

        img1 = tf.image.resize_with_pad(np.expand_dims(img1, axis=0), 192,192)
        input_image = tf.cast(img1, dtype=tf.float32)

        # Setup input and output 
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Make predictions 
        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

        # Rendering 
        draw_connections(img, keypoints_with_scores, EDGES, 0.1)
        draw_keypoints(img, keypoints_with_scores, 0.1)

        frame[indexes[1,i]:indexes[3,i], indexes[0,i]:indexes[2,i]] = img
    return frame

In [15]:
cap = cv2.VideoCapture(0)
#RUN ALL FUNCTIONS IN THIS NOTEBOOK BEFOREHAND!!!!

#cap = cv2.VideoCapture("C:\DLAV_testing\images to test\Business-people-dancing1.mp4")
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    results = model(img)
    pd = get_info_objs(results)
    imglist,indexes = get_cropped_images(pd,frame)
    img = write_frame_keypoints(imglist,indexes,frame)
    
    cv2.imshow('combination', img)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()